<a href="https://colab.research.google.com/github/Rajnish647/project/blob/main/vision_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install flash_attn
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nv

In [1]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
model_id = "microsoft/Phi-3-vision-128k-instruct"

#model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto")
model = AutoModelForCausalLM.from_pretrained('microsoft/Phi-3-vision-128k-instruct', device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation="eager")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
messages = [
    {"role": "user", "content": "<|image_1|>\ncan you extract table given in an image in proper json format"},
    {"role": "assistant", "content": "The chart displays table in an image extract complete table with each row and column properly in json format"},
    {"role": "user", "content": "Provide data properly in proper format."}
]

In [5]:
image = Image.open('/content/sample_data/Acer Inc_page_164.jpg')

In [6]:
prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")

In [22]:
generation_args = {
    "max_new_tokens": 10000,
    "temperature": 0.0,
    "do_sample": False,
}

In [30]:
generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)
# remove input tokens
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

#print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [6]:
def get_text(img):
  image = Image.open(img)
  prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")

  generation_args = {
      "max_new_tokens": 10000,
      "temperature": 0.0,
      "do_sample": False,
  }

  generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

  # remove input tokens
  generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
  response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  print(response)
  return 1

In [7]:
directory = '/content/sample_data/data'

In [ ]:
for img in os.listdir(directory):
  #print(os.path.join(directory, img))
  img = os.path.join(directory, img)
  get_text(img)
  print("=============================================")


[
  {
    "No": 1,
    "Descriptions": "HQ Packages NOC and On-site maintenance",
    "Qty": 1,
    "No. of Month": 3,
    "Monthly Charge (per store)": 330.00,
    "Total Monthly Charge (RM)": 330.00,
    "Amount (RM)": 990.00,
    "a. NOC Monitoring": {
      "24 x 7 x 365 days": "2 connectivity Networks @ Stores",
      "b. Maintenance Assurance (HQ)": {
        "Hardware: 1 x Juniper router",
        "Onsite Maintenance Support": {
          "12 x 7 x 365 days": "Hardware replacement unit provided by Customer/ TM Duration: 1st Apr - 30th Jun 2023"
        }
      }
    }
  },
  {
    "No": 2,
    "Descriptions": "DR Packages NOC and On-site maintenance",
    "Qty": 1,
    "No. of Month": 3,
    "Monthly Charge (per store)": 330.00,
    "Total Monthly Charge (RM)": 330.00,
    "Amount (RM)": 990.00
  }
]

